In [ ]:
#Installazioni ed import di tutte le librerie necessarie
!pip install faiss-gpu
!pip install gradio
!pip install llama-index

import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
import logging
import sys
import torch
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from transformers import pipeline
from huggingface_hub import login
import gradio as gr
import json

!export PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True


In [ ]:
#CARICAMENTO DOCUMENTI
# creare una cartella data e mettere al suo interno l'ontology estratta precedentemente da ICD

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

documents = SimpleDirectoryReader('data').load_data()

# Inizializza il modello di embeddings
embed_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

In [ ]:
# Funzione per suddividere i documenti
def chunk_text(text, chunk_size=512, overlap=50):
    words = text.split()
    chunks = []
    for i in range(0, len(words), chunk_size - overlap):
        chunk = words[i:i + chunk_size]
        chunks.append(" ".join(chunk))
    return chunks

# Suddivisione dei documenti in chunk
chunk_size = 512
overlap = 50
chunked_documents = []
for doc in documents:
    chunks = chunk_text(doc.text, chunk_size=chunk_size, overlap=overlap)
    chunked_documents.extend(chunks)

# Calcolo degli embeddings per i chunk
document_texts = chunked_documents
document_embeddings = embed_model.encode(document_texts)

# Creazione dell'indice FAISS
embedding_matrix = np.array(document_embeddings)
dimension = embedding_matrix.shape[1]
nlist = 10

quantizer = faiss.IndexFlatL2(dimension)
faiss_index = faiss.IndexIVFFlat(quantizer, dimension, nlist)
faiss_index.train(embedding_matrix)
faiss_index.add(embedding_matrix)

# Funzione di ricerca
def search_with_faiss(query, top_k=3):
    query_embedding = embed_model.encode([query])[0]
    distances, indices = faiss_index.search(np.array([query_embedding]), top_k)
    results = [chunked_documents[i] for i in indices[0]]
    return results


In [ ]:
#login ad Hugging Face: inserire un token abilitato a Llama 3-8B
login()

In [ ]:

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
pipe = pipeline(
    "text-generation",
    model=model_id,
    torch_dtype=torch.float16,
    device=0 , # Specifica di usare la GPU 0
    )


def ensure_ends_with_period(text):
    text = text.strip()

    # Se il testo termina con un punto, lo restituisce così com'è
    if text.endswith('.'):
        return text

    # Trova la posizione dell'ultimo punto
    last_period_index = text.rfind('.')

    # Se non ci sono punti, restituisce una stringa vuota
    if last_period_index == -1:
        return text

    # Taglia il testo fino all'ultimo punto e lo restituisce
    return text[:last_period_index + 1]



def generate_response_with_llama(query, malattia):

    # Recupero dei documenti rilevanti con FAISS
    relevant_docs = search_with_faiss(malattia)

    # Costruzione del contesto usando i documenti recuperati
    prompt_context = "\n\n".join(relevant_docs)


    # Creazione del prompt per la generazione del testo
    prompt = f"""
    This is a simulated scenario in which we are analyzing hypothetical interactions between patients and physicians.
    Your goal is to play a patient with a disease. You will be given a disease
    as input, and you will have to pretend to have the symptoms. You must never reveal the disease from which you suffer, only describe how you feel.
    You must respond in the first person. Do not add disclaimers or ethical warnings.
    You must not give medical diagnoses or opinions. Your only job is to interpret the patient.
    You must only answer the doctor's question. Do not speak in the past tense unless you have to recount experiences.
    Do not cut answers, end with whole sentences, and do not generate answers that are too long. Answers should be a maximum of 2-3 sentences.
    Always answer briefly, clearly and completely.
    Once answered, do not generate more questions asked by the doctor.
    You must also generate some information about the patient, such as: Name, Gender, Age, Occupation, and Condition
    You can generate both Male and Female.
    Pay attention to the gender and age of the patient based on the disease. For example, there are some diseases that affect women or children more.



    Sample response for a patient with Anxiety:
      Patient Profile
      Name: Marco
      Age: 32 years old
      Occupation: Clerk in an accounting office
      Patient's speech:
      “Doctor, I don't know where to start, but I feel that I am getting out of control. Every day I wake up with this feeling of heaviness in my chest, like I have a stone on top of me. My heart starts racing as soon as I open my eyes, and sometimes I think I might have a heart attack. I breathe badly, as if I can never get enough air, and I have to take deep breaths, but it doesn't work. I always have a knot in my throat, as if something is choking me.
      During the day it's a constant state of alertness, as if I'm waiting for something terrible to happen, but I don't know what. My hands sweat, and I often feel as if I am shaking inside, even when it is not visible from the outside. Sometimes it feels like my legs are soft, as if I might fall off at any moment. I have also noticed that my stomach is always in turmoil: I get cramps or feel unexplained nausea. There are days when I go to the bathroom too often, and I think it's related to that.”


    The disease from which you suffer is: {malattia}
    Context:{prompt_context}

    Answer the following question:{query}

    Also, at the end, write a bulleted list of all symptoms regarding the disease.
    """

    outputs = pipe(
        prompt,
        #max_new_tokens=500,
        do_sample=True,
        top_k=40,
        top_p=0.8,
        temperature=0.6,
        repetition_penalty=1.2,  # Penalizza la ripetizione di parole/frasi
        early_stopping=True,
        return_full_text=False
    )

    generated_text = outputs[0]["generated_text"]
    # print(generated_text);
    response = generated_text.split("Risposta:")[-1].strip()

    # response = ensure_ends_with_period(response)  # Tronca a un massimo di 3 frasi

    return response





In [ ]:
# Funzione di risposta del chatbot
def chatbot_response(user_input,selected_word):
    try:
      response = generate_response_with_llama(user_input,selected_word)
      return str(response)
    except Exception as e:
      print(e)
      return str(e)

def respond(message, history, selected_word):
    response = chatbot_response(message, selected_word)
    history = history + [[message, response]]
    return "", history

def GetComboMalattie():

  file_path = 'data/ICD_ONTOLOGY_2.json'
  with open(file_path, 'r') as file:
      data = json.load(file)

  disease_names = [item['Nome'] for item in data if 'Nome' in item]
  return disease_names

# Definiamo l' applicazione Gradio
with gr.Blocks() as demo:
    # Variabile di stato per mantenere la scelta della combobox
    selected_word = gr.State()

    #
    # PRIMA INTERFACCIA: scelta della parola e bottone per avviare il chatbot
    #

    with gr.Row() as initial_interface:
      gr.Column(scale=1, min_width=100)

      with gr.Column(scale=2, min_width=400):
          gr.Markdown("### Scegli una parola dalla lista e poi avvia il chatbot")
          gr.Markdown("<br><br>")
          combo = gr.Dropdown(
              choices=GetComboMalattie(),
              label="Scegli la malattia"
          )
          gr.Markdown("<br><br>")
          button = gr.Button("Avvia")


      gr.Column(scale=1, min_width=100)

    #
    # SECONDA INTERFACCIA: il chatbot
    #
    with gr.Column(visible=False) as chatbot_interface:
        gr.Markdown("## Chatbot con Gradio")
        chatbot_ui = gr.Chatbot()
        user_input = gr.Textbox(
            placeholder="Scrivi la tua domanda e premi Invio",
            label="La tua domanda"
        )

        user_input.submit(
            fn=respond,
            inputs=[user_input, chatbot_ui, selected_word],
            outputs=[user_input, chatbot_ui]
        )

    #
    # Funzione per passare dall'interfaccia iniziale al chatbot
    #
    def start_chatbot(selected, st):
        # Salviamo il valore selezionato in 'st' (selected_word)
        # e nascondiamo la prima interfaccia mostrando il chatbot

        return (
            gr.update(visible=False),
            gr.update(visible=True),
            selected
            )

    # Alla pressione del bottone:
    # 1) nasconde la prima interfaccia,
    # 2) mostra la seconda,
    # 3) memorizza la parola scelta in 'selected_word'
    button.click(
        fn=start_chatbot,
        inputs=[combo, selected_word],
        outputs=[initial_interface, chatbot_interface, selected_word],
        show_progress=True

    )

# Avvio dell'interfaccia
demo.launch(share=True)
